In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchvision import transforms

import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image

import os

In [154]:
class EyeDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.transforms = transforms.Compose([transforms.Resize([24, 24]),
                                              transforms.ToTensor()]) 
        self.img_paths = []
        self.img_labels = []

        # Traverses all subfolders and stores the path and label of each training example
        subfolders = os.listdir(path)
        for folder in subfolders:
            p = os.path.join(path, folder)

            if os.path.isdir(p):                                                    # ignores documentation files
                imgs = os.listdir(p)

                for file_name in imgs:
                    self.img_paths.append(os.path.join(p, file_name))               # append image path
                    self.img_labels.append(int(file_name.split("_")[4]))            # append eye state label
    
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB") 
        img = self.transforms(img)
        label = self.img_labels[idx]

        return (img, label)



In [156]:
# Create Dataset object
dataset = EyeDataset(".\MRL")       # Pass in root folder of dataset (only keep subfolders, delete all other )

# Random split (60-20-20)
n = len(dataset)
n_train = int(0.6 *n)
n_valid = int(0.2 *n)
n_test = n - n_train - n_valid

[train_set, valid_set, test_set] = torch.utils.data.random_split(dataset, [n_train, n_valid, n_test])